In [ ]:
import numpy as np
import pandas as pd
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, VotingRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

# ----------------------------
# Função 1: Pré-processamento e Feature Engineering
# ----------------------------
def preprocess_data(df, target='Carga_Traçao', drop_cols=None):
    if drop_cols is None:
        drop_cols = ['ID', 'Argila', 'Areia', 'Silte', 'Torque Max. (kgf.m)',
                     'E(Mpa)', 'Nspt_qc_product']
    df = df.drop(columns=drop_cols, errors='ignore')

    # Normalizar alvo
    scaler_y = MinMaxScaler()
    df[target] = scaler_y.fit_transform(df[[target]])

    # Interpolação de Nspt com profundidade
    mask = df['Nspt'].notnull() & df['Deph(m)'].notnull()
    if mask.sum() > 1:
        df['Nspt'] = np.interp(df['Deph(m)'], df['Deph(m)'][mask], df['Nspt'][mask])

    # Engenharia de atributos
    df['Nspt_qc_ratio'] = df['Nspt'] / (df['qc(Mpa)'] + 1e-8)
    df['Depth_fs_product'] = df['Deph(m)'] * df['fs(Mpa)']
    df['Nspt_carga_product'] = df['Nspt'] * df[target]
    df['log_Nspt'] = np.log1p(df['Nspt'])

    # Winsorização
    Q1 = df['Nspt_carga_product'].quantile(0.25)
    Q3 = df['Nspt_carga_product'].quantile(0.75)
    IQR = Q3 - Q1
    df['Nspt_carga_product'] = df['Nspt_carga_product'].clip(Q1 - 1.5 * IQR, Q3 + 1.5 * IQR)

    df['log_Nspt_carga_product'] = np.log1p(df['Nspt_carga_product'])

    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Imputação
    numeric_cols = df.select_dtypes(include=np.number).columns
    imputer = IterativeImputer(random_state=42)
    df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

    # Corrigir valores físicos
    non_negative_cols = ['Deph(m)', 'Nspt', 'qc(Mpa)', 'fs(Mpa)',
                         'Depth_fs_product', 'Nspt_qc_ratio']
    df[non_negative_cols] = df[non_negative_cols].clip(lower=0)

    # Normalização das features
    features_to_normalize = ['Deph(m)', 'Nspt', 'qc(Mpa)', 'fs(Mpa)',
                             'Depth_fs_product', 'Nspt_qc_ratio', 'log_Nspt_carga_product']
    scaler_X = MinMaxScaler()
    df[features_to_normalize] = scaler_X.fit_transform(df[features_to_normalize])

    return df, scaler_X, scaler_y

# ----------------------------
# Função 2: Treinar modelo
# ----------------------------
def train_model(df, target='Carga_Traçao'):
    X = df.drop(columns=[target])
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = VotingRegressor(estimators=[
        ('xgb', XGBRegressor(n_estimators=500, learning_rate=0.05, max_depth=6, random_state=42)),
        ('rf', RandomForestRegressor(n_estimators=300, max_depth=10, random_state=42))
    ])

    pipeline = Pipeline([('model', model)])
    pipeline.fit(X_train, y_train)

    y_pred = pipeline.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f'\n🔍 Avaliação do Modelo:')
    print(f'MSE: {mse:.4f}')
    print(f'R²: {r2:.4f}')

    return pipeline, X.columns

# ----------------------------
# Função 3: Prever novo valor
# ----------------------------
def predict_new_case(pipeline, new_data, feature_names, scaler_X, scaler_y):
    # Recriar features no novo dado
    new_data['Nspt_qc_ratio'] = new_data['Nspt'] / (new_data['qc(Mpa)'] + 1e-8)
    new_data['Depth_fs_product'] = new_data['Deph(m)'] * new_data['fs(Mpa)']
    new_data['Nspt_carga_product'] = 0
    new_data['log_Nspt'] = np.log1p(new_data['Nspt'])
    new_data['log_Nspt_carga_product'] = np.log1p(new_data['Nspt_carga_product'])

    # Normalizar features
    features_to_normalize = ['Deph(m)', 'Nspt', 'qc(Mpa)', 'fs(Mpa)',
                             'Depth_fs_product', 'Nspt_qc_ratio', 'log_Nspt_carga_product']
    new_data[features_to_normalize] = scaler_X.transform(new_data[features_to_normalize])

    # Previsão
    pred_scaled = pipeline.predict(new_data[feature_names])
    pred_real = scaler_y.inverse_transform(pred_scaled.reshape(-1, 1))
    print(f'\n📌 Previsão da Carga: {pred_real.flatten()[0]:.2f} kN')
    return pred_real.flatten()[0]

# ----------------------------
# Exemplo de uso do script
# ----------------------------
if __name__ == '__main__':
    df = pd.read_csv('base_dataset3_carga.csv')
    df_proc, scaler_X, scaler_y = preprocess_data(df)
    model, features = train_model(df_proc)

    novo_dado = pd.DataFrame({
        'Deph(m)': [15],
        'Nspt': [25],
        'qc(Mpa)': [12.5],
        'fs(Mpa)': [0.15]
    })

    predict_new_case(model, novo_dado, features, scaler_X, scaler_y)
